# Анализ доходности нефтяной скважины

# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона проводится исследование 500 точек.
- Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
- Один баррель сырья приносит 4500 рублей прибыли.
- Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

# 1. Загрузка и подготовка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

# модель
from sklearn.linear_model import LinearRegression

# метрика
from sklearn.metrics import mean_squared_error

In [2]:
# чтение файла и печать общей информации
df_1 = pd.read_csv('/datasets/geo_data_0.csv')
df_2 = pd.read_csv('/datasets/geo_data_1.csv')
df_3 = pd.read_csv('/datasets/geo_data_2.csv')

df_1.head(2)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.280062
1,2acmU,1.334711,-0.340164,4.36508,73.037750


In [3]:
df_2.head(2)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261


In [4]:
df_3.head(2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697


In [5]:
# id для обучения можели не нужны их можно удалить
df_1 = df_1.drop(['id'], axis=1)
df_2 = df_2.drop(['id'], axis=1)
df_3 = df_3.drop(['id'], axis=1)

In [6]:
# разделение на 2 выборки (обучающая и валидационная)
df_train_1, df_valid_1 = train_test_split(df_1, test_size=0.25, random_state=12345)
df_train_2, df_valid_2 = train_test_split(df_2, test_size=0.25, random_state=12345)
df_train_3, df_valid_3 = train_test_split(df_3, test_size=0.25, random_state=12345)
print('Регион 1')
print('Кол-во строк в обучающей выборке:', df_train_1.shape[0])
print('Кол-во строк в валидационной выборке:', df_valid_1.shape[0])
print('')
print('Регион 2')
print('Кол-во строк в обучающей выборке:', df_train_2.shape[0])
print('Кол-во строк в валидационной выборке:', df_valid_2.shape[0])
print('')
print('Регион 3')
print('Кол-во строк в обучающей выборке:', df_train_3.shape[0])
print('Кол-во строк в валидационной выборке:', df_valid_3.shape[0])

Регион 1
Кол-во строк в обучающей выборке: 75000
Кол-во строк в валидационной выборке: 25000

Регион 2
Кол-во строк в обучающей выборке: 75000
Кол-во строк в валидационной выборке: 25000

Регион 3
Кол-во строк в обучающей выборке: 75000
Кол-во строк в валидационной выборке: 25000


In [7]:
# разделение на обучающие признаки и целевые. Регион 1
features_train_1 = df_train_1.drop(['product'], axis=1)
target_train_1 = df_train_1['product']

features_valid_1 = df_valid_1.drop(['product'], axis=1)
target_valid_1 = df_valid_1['product']

# разделение на обучающие признаки и целевые. Регион 2
features_train_2 = df_train_2.drop(['product'], axis=1)
target_train_2 = df_train_2['product']

features_valid_2 = df_valid_2.drop(['product'], axis=1)
target_valid_2 = df_valid_2['product']

# разделение на обучающие признаки и целевые. Регион 3
features_train_3 = df_train_3.drop(['product'], axis=1)
target_train_3 = df_train_3['product']

features_valid_3 = df_valid_3.drop(['product'], axis=1)
target_valid_3 = df_valid_3['product']


In [8]:
# масштабирование признаков
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

In [9]:
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

In [10]:
scaler.fit(features_train_3[numeric])
features_train_3[numeric] = scaler.transform(features_train_3[numeric])
features_valid_3[numeric] = scaler.transform(features_valid_3[numeric])

# 2. Обучение и проверка модели

In [11]:
line_reg_model_1 = LinearRegression().fit(features_train_1, target_train_1)
predicted_valid_1 = line_reg_model_1.predict(features_valid_1)

mse = mean_squared_error(target_valid_1, predicted_valid_1)
rmse = mse ** 0.5
rmse

37.5794217150813

In [12]:
line_reg_model_2 = LinearRegression().fit(features_train_2, target_train_2)
predicted_valid_2 = line_reg_model_2.predict(features_valid_2)

mse = mean_squared_error(target_valid_2, predicted_valid_2)
rmse = mse ** 0.5
rmse

0.8930992867756158

In [13]:
line_reg_model_3 = LinearRegression().fit(features_train_3, target_train_3)
predicted_valid_3 = line_reg_model_3.predict(features_valid_3)

mse = mean_squared_error(target_valid_3, predicted_valid_3)
rmse = mse ** 0.5
rmse

40.02970873393434

Наименьший уровень ошибки у второй модели по втиорому региону. Точность предсказаний самая высокая.

# 3. Подготовка к расчёту прибыли

Для решений задачи запишем параметры в переменных.

In [14]:
# запишем условия в переменных
coutn_dots_explore = 500
total_budget = 10000000000
cost_one_oilwell = 50000000
profit_per_barrel = 4500
risk = 0.025

max_oilwell_count = total_budget // cost_one_oilwell # рассичтаем максимальлное колличество скважин
print(f'Бюджета хватит на разработку {max_oilwell_count} месторождений.')

Бюджета хватит на разработку 200 месторождений.


Посчитаем минимальный объём месторождения для окупаетости бурения скважины.

In [15]:
min_product = cost_one_oilwell // profit_per_barrel
print(f'Минимальный запас месторождения для окупаемости бурения скважины: {min_product} баррелей.')

Минимальный запас месторождения для окупаемости бурения скважины: 11111 баррелей.


In [16]:
print("Средний запас в первом регионе: {:.2f} тысячи баррелей".format(df_1['product'].mean()))
print("Средний запас во втором регионе: {:.2f} тысяч баррелей".format(df_2['product'].mean()))
print("Средний запас в тетьем регионе: {:.2f} тысяч баррелей".format(df_3['product'].mean()))

Средний запас в первом регионе: 92.50 тысячи баррелей
Средний запас во втором регионе: 68.83 тысяч баррелей
Средний запас в тетьем регионе: 95.00 тысяч баррелей


### Вывод:

Чтобы бурение скважине не было убыточным, объем месторожения должен быть не менее 11,1 тыс. баррелей.
На основание средниих значений по объемам запасов, можно сделать выводы, что в среднем во всех регионах добыча нефти будет прибыльной.

In [17]:
# функция для расчета прибыли месторожения
def profit_oilwell(predicted_target):
    max_predicted = (pd.Series(predicted_target)
                     .sort_values(ascending=False)
                     .head(max_oilwell_count)
                    )
    total_profit = (max_predicted.sum() * 1000 * profit_per_barrel) - (cost_one_oilwell * len(max_predicted))
    return total_profit

Функция вычисляет выручку из 200 лучших месторождений.

# 4. Расчёт прибыли и рисков 

In [18]:
# проверим работу функции
print("Прибыль 200 месторождений в регионе 1: {:,.2f}".format(profit_oilwell(predicted_valid_1)))
print("Прибыль 200 месторождений в регионе 2: {:,.2f}".format(profit_oilwell(predicted_valid_2)))
print("Прибыль 200 месторождений в регионе 3: {:,.2f}".format(profit_oilwell(predicted_valid_3)))

Прибыль 200 месторождений в регионе 1: 129,960,488,774.65
Прибыль 200 месторождений в регионе 2: 114,857,120,519.74
Прибыль 200 месторождений в регионе 3: 123,217,543,962.43


Первый регион показывает максимальный показатель.

In [19]:
def bootstrap_profit(data):
    state = np.random.RandomState(42)
    values = []
    data = pd.Series(data)
    for i in range(1000):
        subsample = data.sample(n=500, replace=True, random_state=state)
        values.append(profit_oilwell(subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = len(values[values < 0]) / len(values)
    
    print("Нижняя граница доверительного интервала: {:,.2f} руб.".format(lower))
    print("Верхняя граница доверительного интервала: {:,.2f} руб.".format(upper))
    print("Средняя прибыль: {:,.2f} руб.".format(mean))
    print("Риск убытков: {:,.2f}.".format(risk))

In [20]:
print('Регион 1')
bootstrap_profit(predicted_valid_1)
print('')
print('Регион 2')
bootstrap_profit(predicted_valid_2)
print('')
print('Регион 3')
bootstrap_profit(predicted_valid_3)

Регион 1
Нижняя граница доверительного интервала: 91,491,440,947.03 руб.
Верхняя граница доверительного интервала: 95,761,755,179.74 руб.
Средняя прибыль: 93,593,211,744.40 руб.
Риск убытков: 0.00.

Регион 2
Нижняя граница доверительного интервала: 90,643,000,105.11 руб.
Верхняя граница доверительного интервала: 98,613,846,043.62 руб.
Средняя прибыль: 94,498,416,155.79 руб.
Риск убытков: 0.00.

Регион 3
Нижняя граница доверительного интервала: 90,843,941,436.41 руб.
Верхняя граница доверительного интервала: 94,644,572,555.43 руб.
Средняя прибыль: 92,750,653,715.98 руб.
Риск убытков: 0.00.


### Вывод:

Средняя выручка максимальлная во втором регионе, но с вероятность 95% в перовом регионе выручка составит 91 млрд., а во втором 90 млрд. Таким образом первый регион более перспективный для разработки.

Все три региона показывают хороший уровень риска (0.00), что соответсвует заявленным требованиям.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован